In [ ]:
from azureml.core import Workspace, Experiment, Environment, ScriptRunConfig, Dataset
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.pipeline.core import Pipeline, PipelineParameter
from azureml.data import OutputFileDatasetConfig
from azureml.pipeline.steps import PythonScriptStep
from azureml.widgets import RunDetails
from azureml.core.datastore import Datastore

workspace = Workspace.from_config()

In [ ]:
#datastore = Datastore(workspace, "demostore")
datastore = workspace.get_default_datastore()

input_data = Dataset.File.from_files(datastore.path('mlops_timeseries/mlops_forecast_data2013.csv')).as_named_input('input_ds').as_mount()
inout = OutputFileDatasetConfig(destination=(datastore, 'inout/'))

In [ ]:
aml_compute = 'demo-cpucluster1'  # <== Compute Cluster Name 
source_directory = 'scripts'
model_name = "arima-model-v3"

In [ ]:
run_config_step1 = RunConfiguration()
run_config_step1.target = aml_compute
run_config_step1.environment.python.conda_dependencies = CondaDependencies(conda_dependencies_file_path='arima-env.yml')

In [ ]:
run_config_step2 = RunConfiguration()
run_config_step2.target = aml_compute
run_config_step2.environment.python.conda_dependencies = CondaDependencies(conda_dependencies_file_path='deploy-env.yml')

In [ ]:
step1 = PythonScriptStep(name='train',
                         script_name='train.py', 
                         source_directory=source_directory,
                         arguments =[
                                     '--model_name', model_name, 
                                     '--input', input_data, 
                                     '--output', inout],
                         runconfig=run_config_step1,
                         allow_reuse=False)

In [ ]:
step2 = PythonScriptStep(name='registerAndDeploy',
                         script_name='registerAndDeploy.py', 
                         source_directory=source_directory,
                         arguments =['--input', inout.as_input(name='inout')],
                         runconfig=run_config_step2,
                         allow_reuse=False)

In [ ]:
pipeline_deploy = Pipeline(workspace=workspace, steps=[step1, step2], description='Pipeline Arima v3')

In [ ]:
pipeline_deploy.validate()

In [ ]:
pipeline_run = Experiment(workspace, 'pipeline_deploy_v3').submit(pipeline_deploy, regenerate_outputs=True)

In [ ]:
RunDetails(pipeline_run).show()

In [ ]:
%%time
pipeline_run.wait_for_completion(show_output=True)

## Publish Pipeline

In [ ]:
from azureml.pipeline.core import PublishedPipeline
from azureml.core.authentication import InteractiveLoginAuthentication
import requests

In [ ]:
published_pipeline_deploy = pipeline_deploy.publish(name="Pipeline_arima_v3", description="Pipeline arima v3", continue_on_step_failure=True)
published_pipeline_deploy

In [ ]:
pipeline_id = published_pipeline_deploy.id  # set pipeline id
print(pipeline_id)
published_pipeline = PublishedPipeline.get(workspace, pipeline_id) # retrieve pipeline object with the id
endpoint1 = published_pipeline.endpoint
print(endpoint1)

In [ ]:
auth = InteractiveLoginAuthentication()
aad_token = auth.get_authentication_header()

response = requests.post(endpoint1, 
                         headers=aad_token, 
                         json={"ExperimentName": "Pipeline_call_arima_v3",
                               "RunSource": "SDK"})

In [ ]:
try:
    response.raise_for_status()
except Exception:    
    raise Exception('Received bad response from the endpoint: {}\n'
                    'Response Code: {}\n'
                    'Headers: {}\n'
                    'Content: {}'.format(rest_endpoint, response.status_code, response.headers, response.content))

run_id = response.json().get('Id')
print('Submitted pipeline run: ', run_id)

In [ ]:
#response.json()